In [44]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
class SeleniumClient(object):
    def __init__(self):
        #Initialization method. 
        self.chrome_options = webdriver.ChromeOptions()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-setuid-sandbox')
        self.chrome_options.add_argument("start-maximized")
        self.chrome_options.add_argument("disable-infobars")
        self.chrome_options.add_argument("--disable-extensions")

        # you need to provide the path of chromdriver in your system
    

        chromedriver = "C:/Users/ManasiAdmin/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
       
        
        self.driver = webdriver.Chrome(chromedriver)
        #driver.get("https://twitter.com/search?q=amex")
        #time.sleep(10)
        #driver.quit()

        
        #self.browser = webdriver.Chrome('C:/Users/ManasiAdmin/chromedriver', options=self.chrome_options)

        self.base_url = 'https://twitter.com/search?q='

    def get_tweets(self, query):
        ''' 
        Function to fetch tweets. 
        '''
        try: 
            self.driver.get(self.base_url+query)
            time.sleep(10)

            body = self.driver.find_element_by_tag_name('body')

            for _ in range(30):
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)

            timeline = self.driver.find_element_by_id('timeline')
            #tweet_nodes = timeline.find_elements_by_css_selector('.tweet-text')
            tweet_nodes = timeline.find_elements_by_css_selector('[id^=\".tweet-text\"]')

            return pd.DataFrame({'tweets': [tweet_node.text for tweet_node in tweet_nodes]})

        except: 
            print("Selenium - An error occured while fetching tweets.")

In [45]:
selenium_client = SeleniumClient()

tweets_df = selenium_client.get_tweets('amex')

Selenium - An error occured while fetching tweets.


In [5]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob 
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    if polarity_scores['neg'] > polarity_scores['pos']:
        return 'negative'
    else:
        return 'positive'

def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    # set sentiment 
    if analysis.sentiment.polarity >= 0:
        return 'positive'
    else: 
        return 'negative'

In [10]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text
# We are keeping cleaned tweets in a new column called 'tidy_tweets'
tweets_df['tidy_tweets'] = np.vectorize(remove_pattern)(tweets_df['tweets'], "@[\w]*: | *RT*")

NameError: name 'np' is not defined